# **Cvičení 3: Jak se správně trefit do terče**
---

V tomto cvičení se seznámíme s optimalizačními úlohami. Konkrétně si představíme metodu nejmenších čtverců a ukážeme si, jak ji lze využít k nalezení optimálního míření pro zásah terče.

---

*Následující knihovny a funkce budeme v průběhu cvičení používat:*


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numba

*Toto jsou funkce vytvořené v předchozím cvičení, které využijeme pro simulace střelby na terč:*

In [2]:
@numba.jit
def model_pohybu(beta, gamma, rho, alpha, wx, wy):
    """
    Simuluje pohyb střely a vrací souřadnice dopadu na rovině terče.

    Parametry:
    beta : float
        Úhel vychýlení oproti míření na střed v ose z (v stupních).
    gamma : float
        Úhel vychýlení oproti míření na střed v ose y (v stupních).
    rho : float
        Počáteční rychlost střely.
    alpha : float
        Koeficient odporu vzduchu.
    wx : float
        Rychlost větru ve směru osy x.
    wy : float
        Rychlost větru ve směru osy y.

    Návratové hodnoty:
    z : float
        Výška dopadu střely na terči (osa z).
    y : float
        Horizontální odchylka dopadu střely na terči (osa y).
    """
    # Inicializace konstant a parametrů
    g = 9.81
    m = 0.00051
    dt = 0.01
    n_max = 100
    vzdalenost_terce = 50
    r0 = np.array([0, 0, 0])

    # Výpočet úhlů pro transformaci počáteční rychlosti
    phi = np.pi / 2 - np.deg2rad(beta)
    theta = np.deg2rad(gamma)

    # Výpočet počáteční rychlosti
    v0 = rho * np.array([np.sin(phi) * np.cos(theta), np.sin(phi) * np.sin(theta), np.cos(phi)])

    # Inicializace polí pro trajektorii, rychlost a zrychlení
    r = np.zeros((n_max, 3))
    v = np.zeros((n_max, 3))
    a = np.zeros((n_max, 3))

    # Inicializace vektorů tíhového zrychlení a rychlosti větru
    g_vec = np.array([0, 0, -g])
    w_vec = np.array([wx, wy, 0])

    # Počáteční podmínky
    r[0] = r0
    v[0] = v0

    for i in range(1, n_max):
        a[i - 1] = g_vec - alpha / m * np.linalg.norm(v[i - 1] - w_vec) * (v[i - 1] - w_vec)
        v[i] = v[i - 1] + a[i - 1] * dt
        r[i] = r[i - 1] + (v[i - 1] + v[i]) / 2 * dt
        if r[i, 0] > vzdalenost_terce:  # Ukončení při zásahu terče
            break

    # Výpočet dopadu na terč
    j = i - 1
    y = r[j, 1] + (r[i, 1] - r[j, 1]) / (r[i, 0] - r[j, 0]) * (vzdalenost_terce - r[j, 0])
    z = r[j, 2] + (r[i, 2] - r[j, 2]) / (r[i, 0] - r[j, 0]) * (vzdalenost_terce - r[j, 0])
    return z, y


def vystrel(beta, gamma):
    """
    Simuluje výstřel střely s pevnými parametry.

    Parametry:
    beta : float
        Úhel vychýlení oproti míření na střed v ose z (v stupních).
    gamma : float
        Úhel vychýlení oproti míření na střed v ose y (v stupních).

    Návratové hodnoty:
    tuple
        Souřadnice dopadu střely (z, y).
    """
    # Skryté parametry
    return model_pohybu(beta, gamma, 200, 1.7e-5, -5, 3)

---

## Střelec matematik

Matematik si pořídil vzduchovku a rozhodl se naučit střílet. Protože věří, že si vše dokáže spočítat, míří rovnou na terč vzdálený 50 metrů.


### První čtyři střely

Na rozehřátí se rozhodl vystřelit první čtyři střely. Zaznamenal si úhly náklonu zbraně (ve stupních) i pozice zásahů na "terči". Ano, v prvních třech případech to byla zeď, na které byl terč pověšen, ale změřit to šlo.


In [3]:
# Úhly hlavně ve stupních oproti výstřelu přímo na terč
# Úhel náklonu v ose z (kladné hodnoty směřují nahoru).
beta = np.array([1, 2.5, 1.5, 1.8])
# Úhel náklonu v ose y (kladné hodnoty směřují doprava).
gamma = np.array([-1, -2, -1.5, -1.7])

# Naměřené souřadnice zásahu terče v metrech
# Souřadnice zásahu v ose z (výška zásahu).
z = np.array([-0.91936, 0.44877, -0.46353, -0.18994])
# Souřadnice zásahu v ose y (horizontální odchylka).
y = np.array([0.60353, -0.30889, 0.14654, -0.03597])

To matematikovi stačilo. Vzal tužku a papír, něco si načmáral, spokojeně pokýval hlavou, vzal pušku a vystřelil.

Téměř na střed... jen o milimetr vedle.

Nespokojeně zabrblal a šel domů. Tam vše naťukal do počítače, výsledek si opsal na papír...

Druhý den se vrátil a vystřelil jednu střelu. Dokonalý střed! Tak přesně se to ještě nikomu nepovedlo.

Spokojeně se usmál, prodal vzduchovku s tím, že už má vše vyřešené, a šel domů.

**Zvládneme to i my?**

---


# Popis úlohy

Naším úkolem je určit ideální nastavení zbraně (úhly $\beta$ a $\gamma$)
tak, abychom trefili bod $(0, 0)$ na terči. 

- $\beta$ je úhel vychýlení vůči míření na střed terče v ose $z$ ve stupních (kladné hodnoty odpovídají směru nahoru).
- $\gamma$ je úhel vychýlení  vůči míření na střed terče v ose $y$ (kladné hodnoty odpovídají směru doprava).
  
K dispozici máme množinu $N$ měření, kde každé měření obsahuje dvojici vstupních úhlů $(\beta, \gamma)$ a odpovídající souřadnice zásahu $(z, y)$.

**Jak takovou úlohu vyřešit?**

- Hledáme nějakou kombinaci (funkci) vstupních parametrů $\beta$ a $\gamma$, která nám umožní predikovat zásah do terče.
- Pokud takovou funkci najdeme, můžeme ji využít k určení ideálních úhlů pro zásah do bodu $(0, 0)$.

**Jakou funkci použít?**

Můžeme se podívat jak vypadají grafy závislostí měření na vstupních parametrech $\beta$ a $\gamma$.


In [ ]:
# grafy závislostí zásahu terče na úhlech beta a gamma
fig, axs = plt.subplots(2, 2, figsize=(8, 6))

# Plot z vs beta
axs[0, 0].plot(beta, z, 'o')
axs[0, 0].set_xlabel('beta (degrees)')
axs[0, 0].set_ylabel('z (meters)')
axs[0, 0].set_title('z vs beta')
axs[0, 0].grid(True)

# Plot z vs gamma
axs[0, 1].plot(gamma, z, 'o')
axs[0, 1].set_xlabel('gamma (degrees)')
axs[0, 1].set_ylabel('z (meters)')
axs[0, 1].set_title('z vs gamma')
axs[0, 1].grid(True)

# Plot y vs beta
axs[1, 0].plot(beta, y, 'o')
axs[1, 0].set_xlabel('beta (degrees)')
axs[1, 0].set_ylabel('y (meters)')
axs[1, 0].set_title('y vs beta')
axs[1, 0].grid(True)

# Plot y vs gamma
axs[1, 1].plot(gamma, y, 'o')
axs[1, 1].set_xlabel('gamma (degrees)')
axs[1, 1].set_ylabel('y (meters)')
axs[1, 1].set_title('y vs gamma')
axs[1, 1].grid(True)

plt.tight_layout()
plt.show()

**V podstatě všechna vykreslená data lze proložit přímkou. Tedy zdá se, že lineární aproximace nebude příliš špatná.**

## Formulace lineárního modelu

- Použijeme tedy tzv. lineární model. Vzhledem předchozím grafům a pouze malému počtu měření (4 testovací střely) je to pravděpodobně nejlepší možná volba.
- Lineární model předpokládá, že výsledek závisí lineárně na vstupních parametrech. Například přímka je lineární funkce $$f(x) = a \cdot x + b.$$ 

V našem případě máme dva výstupy, což znamená, že budeme pracovat se dvěma lineárními modely:
$$
\begin{aligned}
    z &= a_{1} \cdot \beta + a_{2} \cdot \gamma + a_3, \\
    y &= b_{1} \cdot \beta + b_{2} \cdot \gamma + b_3.
\end{aligned}
$$
Zde $a_{1}, a_{2}, a_3, b_{1}, b_{2}, b_3$ jsou parametry modelu, které budeme určovat.



**Jak určit parametry modelu?**

## Metoda nejmenších čtverců

Pro určení parametrů použijeme metodu nejmenších čtverců (známe z přednášky!). Naším cílem je minimalizovat chybu mezi skutečnými měřeními a hodnotami predikovanými modelem. Jelikož máme dva modely, budeme určovat dvě sady parametrů:
$$
\mathbf{a} = \begin{bmatrix} a_1 \\ a_2 \\ a_3 \end{bmatrix}, \quad \mathbf{b} = \begin{bmatrix} b_1 \\ b_2 \\ b_3 \end{bmatrix}.
$$

### Maticová formulace

Pro $N$ měření zapíšeme data ve vektorovém tvaru:
$$
\mathbf{M} \mathbf{a} = \mathbf{z}, \quad \mathbf{M} \mathbf{b} = \mathbf{y},
$$
kde:
- $\mathbf{z}$ a $\mathbf{y}$ jsou vektory naměřených pozic zásahů $z$ a $y$, 
- $\mathbf{M}$ je matice vytvořená z naměřených hodnot. Obsahuje vstupní úhly a konstantní člen, takže její aplikace umožní získat predikce modelu. Matice $\mathbf{M}$ má tvar:
$$
\mathbf{M} = \begin{bmatrix}
\beta_1 & \gamma_1 & 1 \\
\beta_2 & \gamma_2 & 1 \\
\vdots & \vdots & \vdots \\
\beta_N & \gamma_N & 1
\end{bmatrix}.
$$

Výsledné soustavy rovnic ale bohužel nemusí vždy mít řešení. 

Důležité je si uvědomit, že pokud máme více rovnic než neznámých, nemusí být možné najít přesné řešení. Můžeme se ale pokusit o nalezení "nejlepšího možného" řešení, například takového, které minimalizuje chyby pomocí **metody nejmenších čtverců**.

#### Jak metoda nejmenších čtverců funguje?
Hledáme vektory $\mathbf{a}$ a $\mathbf{b}$, které minimalizují chybu mezi predikcemi modelu a skutečnými měřeními. 

Pokud by existovalo přesné řešení, platilo by
$$
\mathbf{M} \mathbf{a} - \mathbf{z} = \mathbf{0}, \quad \mathbf{M} \mathbf{b} - \mathbf{y} = \mathbf{0}.
$$

My však předpokládáme, že přesné řešení neexistuje. Proto můžeme velikost rozdílu mezi predikcemi modelu a měřeními vyjádřit jako součet kvadrátů jednotlivých složek rozdílu (kvadrát = čtverec)

$$
E_a(\mathbf{a}) = \|\mathbf{M} \mathbf{a} - \mathbf{z}\|^2 = (\mathbf{M} \mathbf{a} - \mathbf{z})^T (\mathbf{M} \mathbf{a} - \mathbf{z}), \\
E_b(\mathbf{b}) = \|\mathbf{M} \mathbf{b} - \mathbf{y}\|^2 = (\mathbf{M} \mathbf{b} - \mathbf{y})^T (\mathbf{M} \mathbf{b} - \mathbf{y}).
$$

#### Kde je minimum chyby?
Funkce $E_a(\mathbf{a})$ a $E_b(\mathbf{b})$ jsou kvadratické (představte si parabolu). To znamená, že mají minimum, tedy bod ze kterého při posunu libovolným směrem chybu zvýšíme. Proto minimum najdeme tak, že je v něm derivace rovná nule (a jelikož máme proměnných mnoho, platí to pro každou z nich => gradient je nulový vektor)

$$
\nabla E_a(\mathbf{a}) = 0, \quad \nabla E_b(\mathbf{b}) = 0.
$$

*Zájemcům o podrobnosti doporučujeme přednášku Petra Vodstrčila ze ŠKOMAMu 2024 na téma [metody nejmenších čtverců](https://am-nas.vsb.cz/skomam/2024/prednasky/Vodstrcil_SKOMAM_2024.pdf).*

Po zderivování chyby dostaneme
$$
\nabla E_a(\mathbf{a}) = 2 \mathbf{M}^T (\mathbf{M} \mathbf{a} - \mathbf{z}) = 0, \quad \nabla E_b(\mathbf{b}) = 2 \mathbf{M}^T (\mathbf{M} \mathbf{b} - \mathbf{y}) = 0.
$$

Což vede na soustavy rovnic pro metodu nejmenších čtverců
$$
\mathbf{M}^T \mathbf{M} \mathbf{a} = \mathbf{M}^T \mathbf{z},
$$
$$
\mathbf{M}^T \mathbf{M} \mathbf{b} = \mathbf{M}^T \mathbf{y}.
$$

Řešením těchto soustav ($\mathbf{a}$ a $\mathbf{b}$) jsou parametry lineárních modelů, které nejlépe odpovídají našim měřením.

---
## Úkol 1

1. Vytvořte matici $\mathbf{M}$ z naměřených hodnot $\beta$ a $\gamma$. Matice $\mathbf{M}$ má následující tvar:
$$
\mathbf{M} = \begin{bmatrix}
\beta_1 & \gamma_1 & 1 \\
\beta_2 & \gamma_2 & 1 \\
\vdots & \vdots & \vdots \\
\beta_N & \gamma_N & 1
\end{bmatrix}.
$$

2. Použijte naměřené souřadnice zásahů jako vektory $\mathbf{z}$ a $\mathbf{y}$:
$$
\mathbf{z} = \begin{bmatrix}
z_1 \\
z_2 \\
\vdots \\
z_N
\end{bmatrix}, \quad 
\mathbf{y} = \begin{bmatrix}
y_1 \\
y_2 \\
\vdots \\
y_N
\end{bmatrix}.
$$

3. Spočtěte soustavy rovnic pro metodu nejmenších čtverců a určete parametry $\mathbf{a}$ a $\mathbf{b}$:
$$
\mathbf{M}^T \mathbf{M} \mathbf{a} = \mathbf{M}^T \mathbf{z}, \quad 
\mathbf{M}^T \mathbf{M} \mathbf{b} = \mathbf{M}^T \mathbf{y}.
$$

**Výstup úkolu:**
- Parametry $\mathbf{a} = \begin{bmatrix} a_1 \\ a_2 \\ a_3 \end{bmatrix}$ a $\mathbf{b} = \begin{bmatrix} b_1 \\ b_2 \\ b_3 \end{bmatrix}$ získané pomocí výpočtů.


In [ ]:
# Úkol 1

######## doplnte kod #######################

# Sestavení matice M
N = len(z)  # Počet měření
M = np.vstack((beta, gamma, np.ones(N))).T  # Matice M obsahující úhly beta, gamma a konstantní člen

# Výpočet parametrů a
a = np.linalg.solve(M.T @ M, M.T @ z)  # Řešení soustavy pro parametry a

# Výpočet parametrů b
b = np.linalg.solve(M.T @ M, M.T @ y)  # Řešení soustavy pro parametry b

##############################################

print('a =', a, 'b =', b)

---

## Jak pomocí modelu určit ideální úhly pro trefení do terče?

Nyní, když máme model, můžeme jej použít k určení ideálních úhlů pro trefení do terče:
- Chceme zasáhnout bod $(0, 0)$, tedy $z = 0$ a $y = 0$.
- Pro náš lineární model to odpovídá soustavě rovnic:
$$
\begin{aligned}
    0 &= a_{1} \cdot \beta + a_{2} \cdot \gamma + a_3, \\
    0 &= b_{1} \cdot \beta + b_{2} \cdot \gamma + b_3.
\end{aligned}
$$
- Tuto soustavu lze maticově zapsat jako:
$$
\mathbf{A} \mathbf{x} = \mathbf{f},
$$
kde: 
$$
\mathbf{A} = \begin{bmatrix}
a_{1} & a_{2} \\
b_{1} & b_{2}
\end{bmatrix},\quad 
\mathbf{x} = \begin{bmatrix} 
\beta \\ 
\gamma 
\end{bmatrix}, \quad 
\mathbf{f} = \begin{bmatrix} 
-a_3 \\ 
-b_3 
\end{bmatrix}.
$$


---
## Úkol 2

Sestavte matice $\mathbf{A}$ a $\mathbf{f}$ na základě parametrů modelu $\mathbf{a}$ a $\mathbf{b}$. 

Poté vyřešte soustavu rovnic:
$$
\mathbf{A} \mathbf{x} = \mathbf{f},
$$
kde $\mathbf{x} = \begin{bmatrix} \beta \\ \gamma \end{bmatrix}$, a určete ideální úhly $\beta$ a $\gamma$ pro trefení do terče.


In [ ]:
# Úkol 2

######## doplnte kod #######################

# Sestavení matice A a vektoru f
A = np.array([[a[0], a[1]], [b[0], b[1]]])  # Koeficienty z parametrů modelu
f = np.array([-a[2], -b[2]])  # Konstantní členy upravené pro pravou stranu rovnice

# Řešení soustavy rovnic pro určení ideálních úhlů
x = np.linalg.solve(A, f)

##############################################

# Výpis výsledků
print(f"Optimální úhly pro střelbu jsou beta = {x[0]:.3f} [°], gamma = {x[1]:.3f} [°]")

A ještě zkontrolujeme kam se trefíme....

In [ ]:
# ověříme, kam střela dopadne
z_trefa, y_trefa = vystrel(x[0], x[1])
print(f"Střela dopadne na souřadnice z = {1000 * z_trefa:.2f} [mm], y = {1000 * y_trefa:.2f} [mm]")

**Jsme tedy na milimetr přesně ve středu terče!**

---

## Lze to udělat lépe?

Asi ano, když se tak ptáme...

**Využijeme znalost modelu z minulého cvičení!**

Výše je uvedena funkce, která modeluje zásahy do terče. Volat ji můžeme takto: 
`model_pohybu(beta, gamma, rho, alpha, wx, wy)`, kde:
- `beta`, `gamma` jsou úhly vychýlení střely od os $z$ a $y$,
- `rho` je úsťová rychlost střely,
- `alpha` je koeficient odporu vzduchu,
- `wx`, `wy` jsou rychlosti větru ve směru os $x$ a $y$.

Funkce vrací vektor $\begin{bmatrix} z \\ y \end{bmatrix}$, který obsahuje souřadnice zásahu.

Ostatní parametry (již součástí modelu) jsou známé nebo snadno ověřitelné:
- hmotnost střely $m = 0.51 \times 10^{-3} \, \mathrm{m}$,
- tíhové zrychlení $g = 9.81 \, \mathrm{m/s^2}$,
- vzdálenost terče $50 \, \mathrm{m}$.

---

## Co je nyní náš model?

- Náš model je nyní složitější, protože simuluje trajektorii střely. Je popsaný nelineární funkcí.
- Parametry modelu jsou $\rho$, $\alpha$, $w_x$, $w_y$, jejich změny ovlivňují výsledek simulace.
- Model můžeme popsat jako parametrizovanou funkci:
$$
\mathbf{f}(\beta, \gamma; \rho, \alpha, w_x, w_y) = \begin{bmatrix} z \\ y \end{bmatrix},
$$
kde $\beta$ a $\gamma$ jsou vstupní úhly a $\rho$, $\alpha$, $w_x$, $w_y$ jsou parametry modelu.

---

## Optimalizace modelu

Chceme najít optimální parametry $\rho$, $\alpha$, $w_x$, $w_y$, aby model co nejlépe odpovídal naměřeným datům. Měřítkem kvality modelu je chyba:
$$
\mathbf{E}(\rho, \alpha, w_x, w_y) = \sum_{i=1}^{N} \left\Vert \mathbf{f}(\beta_i, \gamma_i; \rho, \alpha, w_x, w_y) - \begin{bmatrix} z_i \\ y_i \end{bmatrix} \right\Vert^2,
$$
kde $\left\Vert \cdot \right\Vert^2$ je Euklidovská norma vektoru na druhou. 

Optimální parametry minimalizují chybu:
$$
\rho^*, \alpha^*, w_x^*, w_y^* = \arg \min_{\rho, \alpha, w_x, w_y} \mathbf{E}(\rho, \alpha, w_x, w_y).
$$

---

**Nebojme se!**

I když problém vypadá složitě, existují matematické knihovny, které tento výpočet zvládnou. Například operaci 
$$
\arg \min_{\rho, \alpha, w_x, w_y} \mathbf{E}
$$
dokáže provést funkce `minimize` z knihovny `scipy.optimize`. Stačí, když:
1. Vytvoříme funkci pro výpočet chyby $\mathbf{E}$.
2. Zadáme počáteční odhad parametrů.


---
## Úkol 3

Sestavte funkci pro výpočet chyby $\mathbf{E}$ na základě naměřených dat a použijte ji ve funkci `minimize` k nalezení optimálních parametrů modelu.

### Počáteční odhad parametrů:
- Střelec odhadl úsťovou rychlost své vzduchovky na $160 \, \mathrm{m/s}$.
- Diabolka má tvar kruhu o poloměru $0.003 \, \mathrm{m}$, což znamená, že její plocha je přibližně $2.8 \cdot 10^{-5}\, \mathrm{m^2} $. Hustota vzduchu je asi $1.225\, \mathrm{kg/m^3}$, takže koeficient odporu vzduchu odhaduje na $3.5 \cdot 10^{-5}\, \mathrm{kg/m}$.
- Střelec cítí slabý vítr, který mu naráží na levou tvář. Proto odhaduje $w_x < 0$ a $w_y > 0$, například $w_x = -1 \, \mathrm{m/s}$ a $w_y = 1 \, \mathrm{m/s}$.

### Poznámky:
- Máte k dispozici funkci `model_pohybu(beta, gamma, rho, alpha, wx, wy)`, která vypočítá zásah do terče na základě vstupních úhlů a parametrů modelu.
- Funkce pro výpočet chyby $\mathbf{E}$ musí přijímat jediný argument: vektor parametrů $\mathbf{p} = [\rho, \alpha, w_x, w_y]$.
- Naměřená data (`beta`, `gamma`, `z`, `y`) jsou uložena v globálních proměnných a lze je ve funkci pro chybu $\mathbf{E}$ přímo využít.
- Funkce musí vracet jedno číslo, které představuje celkovou velikost chyby $\mathbf{E}$:
$$
\mathbf{E}(\rho, \alpha, w_x, w_y) = \sum_{i=1}^{N} \left\Vert \mathbf{f}(\beta_i, \gamma_i; \rho, \alpha, w_x, w_y) - \begin{bmatrix} z_i \\ y_i \end{bmatrix} \right\Vert^2,
$$
kde $\left\Vert \cdot \right\Vert^2$ je Euklidovská norma vektoru na druhou.


In [ ]:
# Úkol 3
import scipy.optimize as opt


def E_nelinearni_ctverce(parametry):
    """
    Výpočet chyby E pro zadané parametry modelu.

    Parametry:
    parametry : list nebo ndarray
        Vektor parametrů modelu [rho, alpha, wx, wy].

    Návratová hodnota:
    float
        Celková chyba E mezi predikovanými a naměřenými zásahy.
    """
    ######## doplnte kod #######################
    E = 0
    for i in range(4):  # Pro každé měření spočítáme příspěvek k chybě
        z_trefa, y_trefa = model_pohybu(beta[i], gamma[i], parametry[0], parametry[1], parametry[2], parametry[3])
        E += (z_trefa - z[i])**2 + (y_trefa - y[i])**2  # Součet kvadrátů rozdílů
    ##############################################
    return E


# Optimalizace parametrů pomocí metody Nelder-Mead
res = opt.minimize(E_nelinearni_ctverce, [160, 3.5e-5, -1, 1],
                   method='Nelder-Mead', tol=1e-9, options={'maxiter': 10000})

# Extrakce nalezených parametrů
parametry = res.x
print("Nalezli jsme následující hodnoty parametrů:")
print(f"rho = {parametry[0]:.2f} [m/s],")
print(f"alpha = {parametry[1]:.2e} [kg/m],")
print(f"wx = {parametry[2]:.2f} [m/s],")
print(f"wy = {parametry[3]:.2f} [m/s]")

A skutečné skryté parametry modelu jsou:

$\rho = 200\, \mathrm{m/s}$,

$\alpha = 1.7\cdot 10^{-5}\, \mathrm{kg/m}$,

$w_x = -5 \, \mathrm{m/s}$,

$w_y = 3\, \mathrm{m/s}$

---

## Poslední krok: známe model a parametry... pojďme optimalizovat!

Známe-li model včetně parametrů, můžeme si jej představit jako funkci, která v závislosti na vstupních úhlech $\beta$ a $\gamma$ vypočítá zásah do terče:
$$
\mathbf{f}(\beta, \gamma) = \begin{bmatrix} z \\ y \end{bmatrix}.
$$

Naším cílem je trefit bod $(0, 0)$, což znamená najít takové $\beta$ a $\gamma$, pro které platí:
$$
\mathbf{f}(\beta, \gamma) = \begin{bmatrix} z \\ y \end{bmatrix} = \begin{bmatrix} 0 \\ 0 \end{bmatrix}.
$$

### Optimalizace

Předpokládejme, že přesné řešení nemusí existovat. Proto budeme hledat takové $\beta$ a $\gamma$, které minimalizují chybu:
$$
\mathbf{E}(\beta, \gamma) = \left\Vert \mathbf{f}(\beta, \gamma) - \begin{bmatrix} 0 \\ 0 \end{bmatrix} \right\Vert^2,
$$
kde $\left\Vert \cdot \right\Vert^2$ je Euklidovská norma vektoru na druhou.

Úlohu tedy formálně zapíšeme jako:
$$
\beta^*, \gamma^* = \arg \min_{\beta, \gamma} \mathbf{E}(\beta, \gamma).
$$

### Co udělat:
1. Napište funkci, která vypočítá chybu $\mathbf{E}(\beta, \gamma)$.
2. Použijte funkci `minimize` z knihovny `scipy.optimize` k nalezení optimálních úhlů $\beta^*$ a $\gamma^*$.

**Poznámky:**
- Stejně jako v předchozím úkolu použijte globální proměnnou `parametry`, která obsahuje nalezené hodnoty $\rho^*$, $\alpha^*$, $w_x^*$ a $w_y^*$.
- Použijte funkci `model_pohybu(beta, gamma, rho, alpha, wx, wy)` pro výpočet modelu.


---
## Úkol 4

Najděte optimální úhly $\beta$ a $\gamma$ pro trefení do terče v bodě $(0, 0)$.

### Počáteční odhad:
Pro počáteční odhad můžete použít například:
$$
\beta = 0, \quad \gamma = 0.
$$

### Postup:
1. Definujte funkci pro výpočet chyby $\mathbf{E}(\beta, \gamma)$, která bude vracet hodnotu chyby pro zadané úhly.
2. Použijte funkci `minimize` z knihovny `scipy.optimize` k nalezení optimálních hodnot $\beta^*$ a $\gamma^*$.

**Poznámka:** Při výpočtu využijte dříve nalezené parametry modelu (`parametry`) a funkci `model_pohybu(beta, gamma, rho, alpha, wx, wy)`.


In [ ]:
# Úkol 4

def E_strelba(uhly):
    """
    Výpočet chyby E pro zadané úhly beta a gamma.

    Parametry:
    uhly : list nebo ndarray
        Vektor úhlů [beta, gamma].

    Návratová hodnota:
    float
        Hodnota chyby E, tj. vzdálenost zásahu od bodu (0, 0) na terči.
    """
    ######## doplnte kod #######################
    y, z = model_pohybu(uhly[0], uhly[1], parametry[0], parametry[1], parametry[2], parametry[3])
    sum = (y)**2 + (z)**2  # Kvadráty rozdílů souřadnic zásahu od cíle
    ############################################
    return sum


# Optimalizace úhlů pro minimální chybu
res = opt.minimize(E_strelba, [0, 0], method='Nelder-Mead', tol=1e-9, options={'maxiter': 10000})
optimalni_uhly = res.x

# Výpis nalezených optimálních úhlů
print(f"Optimální úhly pro střelbu jsou beta = {optimalni_uhly[0]:.3f} [°], gamma = {optimalni_uhly[1]:.3f} [°]")

In [ ]:
# ověříme, kam střela dopadne
z_trefa, y_trefa = vystrel(res.x[0], res.x[1])
print(f"Střela dopadne na souřadnice z = {1000 * z_trefa:.2f} [mm], y = {1000 * y_trefa:.2f} [mm]")

**Dokonalá trefa! Doslova ani ne o vlas vedle středu.**  
(Lidský vlas má tloušťku přibližně 0,05–0,1 mm.)


</br>
</br>
</br>
</br>
</br>
</br>

---
---
---
---
# Bonusové materiály:
---


## Mohl to náš ostrostřelec udělat už po dvou střelách?

### Pomocí lineárního modelu:

V předchozích cvičeních jsme viděli, že i takto "jednoduchý" model může být velmi přesný. Co kdybychom ale měli pouze dvě testovací střely? Šlo by to spočítat i tak?

Zkusme se nad tím zamyslet...

S pouhými dvěmi měřeními bychom museli určit 3 parametry, což v případě lineárního modelu vede nejednoznačnému řešení (nebo žádnému). 

### Jak se s tím vypořádat?

Nezbývá než zjednodušit model. Z předchozích výsledků víme, že $z$ téměř výhradně závisí na $\beta$ a $y$ na $\gamma$. Můžeme tedy zkusit použít zjednodušený lineární model:
$$
\begin{aligned}
    z &= a_{1} \cdot \beta + a_3, \\
    y &= b_{2} \cdot \gamma + b_3.
\end{aligned}
$$

### Matice pro zjednodušený model

Pro zjednodušený model budou výsledné matice vypadat takto:
$$
\mathbf{M_a} \mathbf{a} = \mathbf{z}, \quad \mathbf{M_b} \mathbf{b} = \mathbf{y},
$$
kde:
$$
\mathbf{M_a} = \begin{bmatrix}
\beta_1 & 1 \\
\beta_2 & 1
\end{bmatrix}, \quad 
\mathbf{M_b} = \begin{bmatrix}
\gamma_1 & 1 \\
\gamma_2 & 1
\end{bmatrix}.
$$

### Řešení soustavy

S výsledným modelem bude požadované řešení odpovídat soustavě rovnic:
$$
\mathbf{A} \mathbf{x} = \mathbf{f},
$$
kde:
$$
\mathbf{A} = \begin{bmatrix}
a_{1} & 0 \\
0 & b_{2}
\end{bmatrix}, \quad 
\mathbf{x} = \begin{bmatrix} 
\beta \\ 
\gamma 
\end{bmatrix}, \quad 
\mathbf{f} = \begin{bmatrix} 
-a_3 \\ 
-b_3 
\end{bmatrix}.
$$


---
## Úkol B1

Proveďte totéž co v úkolu 1 a 2, ale pouze s redukovaným lineárním modelem a prvními dvěma měřeními.


**Výstup:** 
- Parametry redukovaného modelu $\mathbf{a}$ a $\mathbf{b}$.
- Optimální úhly $\beta$ a $\gamma$ pro trefení do terče s použitím dvou měření.


In [ ]:
# Úkol 1

######## doplnte kod #######################

# Sestavení matic Ma a Mb z prvních dvou měření
N = 2  # Počet použitých měření
Ma = np.vstack((beta[:2], np.ones(N))).T  # Matice pro z
Mb = np.vstack((gamma[:2], np.ones(N))).T  # Matice pro y

# Výpočet parametrů a a b pro redukovaný model
a = np.linalg.solve(Ma.T @ Ma, Ma.T @ z[:2])  # Parametry pro z
b = np.linalg.solve(Mb.T @ Mb, Mb.T @ y[:2])  # Parametry pro y

##############################################

print('a =', a, 'b =', b)

######## doplnte kod #######################

# Sestavení diagonální matice A a vektoru f
A = np.array([[a[0], 0], [0, b[0]]])  # Diagonální matice s parametry modelu
f = np.array([-a[1], -b[1]])  # Vektor pravých stran rovnice

# Výpočet požadovaných úhlů beta a gamma
x = np.linalg.solve(A, f)

##############################################

# Výpis výsledků
print('x =', x)  # Výsledné úhly beta a gamma

In [ ]:
# ověříme, kam střela dopadne
z_trefa, y_trefa = vystrel(x[0], x[1])
print(f"Střela dopadne na souřadnice z = {1000 * z_trefa:.2f} [mm], y = {1000 * y_trefa:.2f} [mm]")

**Stále se trefíme prakticky (méně než 1mm) přesně do středu terče!**

---

## Úkol B2

Co když bychom měli pouze jedno měření? Jak přesně bychom dokázali trefit terč už na druhý pokus?

### Zadání:
1. **Problém:** Jeden testovací výstřel poskytuje pouze jedno měření, což znamená, že použití lineárního modelu nebude možné. 
2. **Řešení:** Použijte druhý přístup – optimalizaci parametrů fyzikálního modelu trajektorie. Využijte dostupná data z jednoho měření a nalezené parametry modelu.

### Postup:
1. **Vstupní data:** 
   - Úhly a polohu zásahu z jednoho měření ($\beta$, $\gamma$, $z$, $y$).
2. **Optimalizace:**
   - Nejprve zrčete parametry trajektorie ($\rho^*$, $\alpha^*$, $w_x^*$, $w_y^*$) pomocí optimalizace.
   - Poté nalezněte optimální úhly $\beta^*$ a $\gamma^*$



In [ ]:
# Úkol B2
import scipy.optimize as opt

# Optimalizace parametrů modelu na základě jednoho měření


def E_nelinearni_ctverce(parametry):
    """
    Výpočet chyby E pro optimalizaci parametrů modelu na základě jednoho měření.

    Parametry:
    parametry : list nebo ndarray
        Vektor parametrů modelu [rho, alpha, wx, wy].

    Návratová hodnota:
    float
        Celková chyba E pro zadané parametry modelu.
    """
    ######## doplnte kod #######################
    E = 0
    i = 0  # Index měření, které použijeme pro optimalizaci
    z_trefa, y_trefa = model_pohybu(beta[i], gamma[i], parametry[0], parametry[1], parametry[2], parametry[3])
    E += (z_trefa - z[i])**2 + (y_trefa - y[i])**2  # Kvadráty rozdílů mezi simulací a měřením
    ##############################################
    return E


# Optimalizace parametrů modelu
res = opt.minimize(E_nelinearni_ctverce, [160, 3.5e-5, -1, 1],
                   method='Nelder-Mead', tol=1e-9, options={'maxiter': 10000})
parametry = res.x  # Nalezené parametry modelu

# Optimalizace úhlů střelby


def E_strelba(uhly):
    """
    Výpočet chyby E pro optimalizaci úhlů střelby.

    Parametry:
    uhly : list nebo ndarray
        Vektor úhlů [beta, gamma].

    Návratová hodnota:
    float
        Hodnota chyby E, tj. vzdálenost zásahu od bodu (0, 0).
    """
    ######## doplnte kod #######################
    y, z = model_pohybu(uhly[0], uhly[1], parametry[0], parametry[1], parametry[2], parametry[3])
    sum = (y)**2 + (z)**2  # Kvadráty rozdílů souřadnic zásahu od cíle
    ############################################
    return sum


# Optimalizace úhlů pro minimální chybu
res = opt.minimize(E_strelba, [0, 0], method='Nelder-Mead', tol=1e-9, options={'maxiter': 10000})
optimalni_uhly = res.x

# Výpis nalezených optimálních úhlů
print(f"Optimální úhly pro střelbu jsou beta = {optimalni_uhly[0]:.3f} [°], gamma = {optimalni_uhly[1]:.3f} [°]")

# Ověření místa dopadu střely
z_trefa, y_trefa = vystrel(res.x[0], res.x[1])
print(f"Střela dopadne na souřadnice z = {1000 * z_trefa:.2f} [mm], y = {1000 * y_trefa:.2f} [mm]")